# Albedo Regression Example

# Imports

In [ ]:
import ROOT
from ROOT import TMVA,TFile, TTree, TCut, TString, TCanvas,  TASImage
import matplotlib.pyplot as plt
import numpy as np

# Plotting raw data

In [ ]:
img=np.genfromtxt('data/Albedo_Map.csv',delimiter=',')
imgplot = plt.imshow(img)
plt.show()

## Declare Factory
Initiate the TMVA library, get the data sample from github, and create a factory to do the regression.

In [ ]:
TMVA.Tools.Instance();

inputFile = TFile.Open("data/dataset.root");
outputFile = TFile.Open("TMVAOutputBDT.root", "RECREATE");

factory = TMVA.Factory("TMVARegression", outputFile,
                      "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" ); 

## Declare DataLoader
Define the features and the target for the regression.

In [ ]:
loader = TMVA.DataLoader("dataset"); 

#Add the feature variables, names reference branches in inputFile ttree
loader.AddVariable("Fast_Map_extra", "", "units", 'F' );
loader.AddVariable("LPFe_Map", "", "units", 'F' );
loader.AddVariable("LPK_Map_extra", "", "units", 'F' );
loader.AddVariable("LPTh_Map", "", "units", 'F' );
loader.AddVariable("LPTi_Map", "", "units", 'F' );
loader.AddVariable("Therm_Map", "", "units", 'F' );

loader.AddTarget( "Albedo_Map" ); # define the target for the regression


## Setup Dataset
Link dataloader to dataset.

In [ ]:
tree=TTree()
inputFile.GetObject("TreeR", tree);

mycuts = TCut(""); # e.g. TCut mycuts = "abs(var1)<0.5";

loader.AddRegressionTree(tree, 1.0);   # link the TTree to the loader, weight for each event  = 1
loader.PrepareTrainingAndTestTree(mycuts,"nTrain_Regression=129600:nTest_Regression=129960:SplitMode=Random:NormMode=NumEvents:!V" );

# Book The Regression Method

Book the method for regression. Here we choose the Boosted Decision Tree model. You have to use gradient boosted trees for regression, hence the BDTG and BoostType=Grad. 

Define the hyperparameters: ntrees, boosttype, shrinkage, and the depth. Also define the loss function you want to use: 'AbsoluteDeviation', 'Huber', or 'LeastSquares'. nCuts determines how finely to look at each feature. Larger values take more time, but you may get more accurate results.

In [ ]:
# Boosted Decision Trees 
factory.BookMethod(loader,TMVA.Types.kBDT, "BDTG","!H:!V:NTrees=200::BoostType=Grad:Shrinkage=0.3:nCuts=20:MaxDepth=4:RegressionLossFunctionBDTG=AbsoluteDeviation");

# Train Method

In [ ]:
factory.TrainAllMethods();

# Test and Evaluate the Model

In [ ]:
factory.TestAllMethods();
factory.EvaluateAllMethods();    

## Gather and Plot the Results
Let's plot the residuals for the BDTG predictions. First, close the output file so that it saves to disk and we can open it without issue. Then get the results on the test set. Finally, plot the residuals.

In [ ]:
%jsroot on
outputFile.Close();
resultsFile = TFile.Open("TMVAOutputBDT.root");
resultsTree = resultsFile.Get("dataset/TestTree"); 
c=TCanvas() ;
resultsTree.Draw("BDTG"); # BDTG is the predicted value, target is the true value
c.Draw()